In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib import image
import random

### Obtain images

In [3]:
# load image files in directory
imagesfiles = [f for f in os.listdir('deepweeds_data/images/')]

images = []
for file in imagesfiles:
    im = image.imread('deepweeds_data/images/{}'.format(file))
    images.append(im)

In [4]:
print(images[1].shape)

(256, 256, 3)


### Obtain labels

In [5]:
arr = np.genfromtxt("deepweeds_data/labels.csv", delimiter=",")
label_values = arr[:,1][1:]
labels = []
for idx, label in enumerate(label_values):
    new_label = np.zeros(9)
    new_label[int(label)]
    labels.append(new_label)

In [6]:
""" Splitting data into training and testing - with a randomly shuffled 80/20 split"""
labels = labels[:len(images)]

train_size = int(np.round(len(labels)*0.80))
train_permuts = random.sample(range(0,len(labels)), train_size)
valid_permuts = np.linspace(0,len(labels)-1, len(labels))

valid_permuts = [int(i) for i in valid_permuts if i not in train_permuts]
x_train, y_train = np.array(images)[train_permuts], np.array(labels)[train_permuts]
x_valid, y_valid = np.array(images)[valid_permuts], np.array(labels)[valid_permuts]

### Load model

In [7]:
import tensorflow as tf
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


CLASSES = 9
IMG_SIZE = (256, 256)
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)

In [8]:
base_model = ResNet50(weights='imagenet', include_top=False)#, input_shape=INPUT_SHAPE)
x = base_model.output
# Add a global average pooling layer
x = GlobalAveragePooling2D(name='avg_pool')(x)
# Add fully connected output layer with sigmoid activation for multi label classification
outputs = Dense((CLASSES), activation='sigmoid', name='fc9')(x)
# Assemble the modified model
model = Model(inputs=base_model.input, outputs=outputs)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

### Augment data as done in the paper

In [9]:
data_augment= ImageDataGenerator(
            rescale=1. / 255,
            fill_mode="constant",
            shear_range=0.2,
            zoom_range=(0.5, 1),
            horizontal_flip=True,
            rotation_range=360,
            channel_shift_range=25,
            brightness_range=(0.75, 1.25))
     
data_augment.fit(x_train)

In [10]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['categorical_accuracy'])
print(len(x_train), len(x_valid))
model.fit(data_augment.flow(np.asarray(x_train[:100]), np.asarray(y_train[:100]), batch_size=1000,
         subset='training'),
         steps_per_epoch=len(x_train) / 1000, epochs=3)